In [32]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
load_dotenv()

True

## Tool Create 

In [60]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/cd25ed03ee32a16a6b93a44c/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [61]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1754870402,
 'time_last_update_utc': 'Mon, 11 Aug 2025 00:00:02 +0000',
 'time_next_update_unix': 1754956802,
 'time_next_update_utc': 'Tue, 12 Aug 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.625}

In [35]:
convert.invoke({"base_currency_value": 10, "conversion_rate": 82.0})

820.0

## Tool Binding 

In [36]:


llm = HuggingFaceEndpoint(
    repo_id="zai-org/GLM-4.5",
    task="text-generation")

model = ChatHuggingFace(llm=llm)

In [37]:
llm_with_tools = model.bind_tools(tools=[get_conversion_factor, convert])

## Tool Calling

In [62]:
messages = [HumanMessage('Convert 100 USD to INR')]

In [63]:
ai_message = llm_with_tools.invoke(messages)
#messages.append(ai_message)
ai_message 

AIMessage(content="\n\nI'll help you convert 100 USD to INR. First, let me get the current conversion rate between USD and INR.\n", additional_kwargs={'tool_calls': [{'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor', 'description': None}, 'id': 'call_phNQnE2V4D8Pi9kTT43oImBd', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 267, 'total_tokens': 385}, 'model_name': 'zai-org/GLM-4.5', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--fa7cd346-8963-4988-9ee1-c3fa2f4d1d97-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_phNQnE2V4D8Pi9kTT43oImBd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 267, 'output_tokens': 118, 'total_tokens': 385})

In [64]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_phNQnE2V4D8Pi9kTT43oImBd',
  'type': 'tool_call'}]

In [47]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)


In [48]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, Give me value of dollar against 100 rupees', additional_kwargs={}, response_metadata={}),
 AIMessage(content="\n\nI'll help you get the conversion factor between INR and USD, and then calculate the value of 100 rupees in dollars.\n", additional_kwargs={'tool_calls': [{'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor', 'description': None}, 'id': 'call_zWntn3X8ezRYjGqkjvqqqxFM', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 281, 'total_tokens': 471}, 'model_name': 'zai-org/GLM-4.5', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2718e940-d28c-4ae9-b022-1c007c5e2eb1-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'call_zWntn3X8ezRYjGqkjvqqqxFM', 'type': 'tool_call'}], usage_metadata={'input_tok

In [49]:
result=llm_with_tools.invoke(messages).content

result

''

In [50]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)